In [ ]:
pip install langchain langgraph langchain_openai langchainhub langsmith duckduckgo-search beautifulsoup4 gradio

In [ ]:
pip install getpass4

In [ ]:
import getpass
open_ai_api_key=getpass.getpass("Enter your open ai api key")

In [ ]:
#using summary node

import functools, operator, requests, os, json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import gradio as gr

os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Research Agents"





llm = ChatOpenAI(model="gpt-4-turbo-preview",api_key=open_ai_api_key)


@tool("internet_search", return_direct=False)
def internet_search(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return results if results else "No results found."

@tool("process_content", return_direct=False)
def process_content(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search, process_content]


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


members = ["Web_Searcher", "Insight_Researcher","Summariser"]
system_prompt = (
    "As a supervisor, your role is to oversee a dialogue between the workers and perform  task as defined below by choosing the appropiate workesr  accordingly"
    " workers: {members}."
    "Task : Based on the input you need to search the web for the topic using relevant search terms , if any insights are provided , make sure to follow them as well."
    "After getting the result based on the seacrh , you need to do an insight research based on the result."
    " Then, in the end you need to make a condensed summary of the content from previous steps, return the summary it as a result"
    " Once all steps are complete,"
    " indicate with 'FINISH'."
)

options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}] }},
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (prompt | llm.bind_functions(functions=[function_def], function_call="route") | JsonOutputFunctionsParser())


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

search_agent = create_agent(llm, tools, "You are a web searcher. Search the internet for information.")
search_node = functools.partial(agent_node, agent=search_agent, name="Web_Searcher")

insights_research_agent = create_agent(llm, tools, 
        """You are a Insight Researcher. Do step by step. 
        Based on the provided content first identify the list of relevant sub topics based on the main topic,
        then search internet for each topic one by one
        and finally find insights for each topic one by one.
        Include only the insights  in the final response
        """)
insights_research_node = functools.partial(agent_node, agent=insights_research_agent, name="Insight_Researcher")


summary_agent = create_agent(llm, tools, "You are an expert content summariser and have a lot of professional experience in the media. Given the content make a condensed summary of the content relevant to the topic. Make sure the summary is worthy of reading and is interesting and engaging and can be posted as a content on any social site.")
summary_node = functools.partial(agent_node, agent=summary_agent, name="Summariser")

workflow = StateGraph(AgentState)
workflow.add_node("Web_Searcher", search_node)
workflow.add_node("Insight_Researcher", insights_research_node)
workflow.add_node("Summariser", summary_node)
workflow.add_node("supervisor", supervisor_chain)


for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()




def run_graph(title,user_insight):
    input_message=title+" "+user_insight
    response = graph.invoke({
        "messages": [HumanMessage(content=input_message)]
    })
    #print(json.dumps(response['messages'][1].content, indent=2))
    print(response)
    return json.dumps(response['messages'][-1].content, indent=2)
    #summary_markdown = response['messages'][1].content
    #summary_text = markdown_to_text(summary_markdown)
    #print(summary_text)
    #return summary_text

#outputs = gr.Textbox(label="outputs")

demo = gr.Interface(fn=run_graph, inputs=["text","text"], outputs=gr.components.Markdown())
demo.launch()


In [ ]:
# Using chroma database


import functools
import operator
import requests
import os
import json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence, TypedDict
import gradio as gr
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser


os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Research Agents"

llm = ChatOpenAI(model="gpt-4-turbo-preview", api_key=open_ai_api_key)


embeddings = OpenAIEmbeddings(api_key=open_ai_api_key)
vector_store = Chroma(embedding_function=embeddings)


@tool("internet_search", return_direct=False)
def internet_search(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r['body'] for r in ddgs.text(query, max_results=3)]
        return results if results else "No results found."

@tool("process_content", return_direct=False)
def process_content(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search, process_content]


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def agent_node(state, agent, name):
    result = agent.invoke(state)
    # Store intermediate result in vector store
    text = result["output"]
    vector_store.add_texts([text])
    return {"messages": [HumanMessage(content=text, name=name)]}


members = ["Web_Searcher", "Insight_Researcher", "Summariser"]
system_prompt = (
    "As a supervisor, your role is to oversee a dialogue between the workers and perform tasks as defined below by choosing the appropriate workers accordingly. "
    "Workers: {members}. "
    "Task: Based on the input, you need to search the web for the topic using relevant search terms. "
    "After getting the result based on the search, you need to do insight research based on the result. "
    "Once all steps are complete, indicate with 'FINISH'."
)

options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}] }},
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (prompt | llm.bind_functions(functions=[function_def], function_call="route") | JsonOutputFunctionsParser())


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

search_agent = create_agent(llm, tools, "You are a web searcher. Search the internet for information.")
search_node = functools.partial(agent_node, agent=search_agent, name="Web_Searcher")

insights_research_agent = create_agent(llm, tools, 
        """You are an Insight Researcher. Do step by step. 
        Based on the provided content, first identify the list of relevant subtopics based on the main topic,
        then search the internet for each topic one by one,
        and finally find insights for each topic one by one.
        Include only the insights in the final response.
        """)
insights_research_node = functools.partial(agent_node, agent=insights_research_agent, name="Insight_Researcher")

summary_agent = create_agent(llm, tools, "You are an expert content summarizer and have a lot of professional experience in the media. Given the content, make a condensed summary of the content relevant to the topic. Make sure the summary is worthy of reading and is interesting and engaging and can be posted as content on any social site.")
summary_node = functools.partial(agent_node, agent=summary_agent, name="Summariser")

workflow = StateGraph(AgentState)
workflow.add_node("Web_Searcher", search_node)
workflow.add_node("Insight_Researcher", insights_research_node)
workflow.add_node("Summariser", summary_node)
workflow.add_node("supervisor", supervisor_chain)


for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()

def generate_summarization_prompt(texts):
    prompt = (
        "You are an expert content summarizer. Given the following texts, create a condensed, "
        "information-dense summary that connects all the subtopics. The summary should be engaging and suitable for posting as content on any social site.\n\n"
        "Texts:\n" + "\n\n".join(texts) + "\n\nSummary:"
    )
    return prompt


def run_graph(title, user_insight):
    input_message = title + " " + user_insight
    response = graph.invoke({
        "messages": [HumanMessage(content=input_message)]
    })


    texts = [doc.page_content for doc in vector_store.similarity_search(input_message, k=10)]


    summarization_prompt = generate_summarization_prompt(texts)


    summary_response = llm.invoke(summarization_prompt)
    parser = StrOutputParser()
    summary_parse_response=parser.invoke(summary_response)
   
    print(summary_parse_response)
    return summary_parse_response

outputs = gr.Textbox(label="outputs")
demo = gr.Interface(fn=run_graph, inputs=["text", "text"], outputs=outputs)
demo.launch()
